<a href="https://colab.research.google.com/github/seelca/more-access/blob/dev/more_access_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#to do next:
#load points from API https://data.bus-data.dft.gov.uk/api/buslocation-openapi/#/SIRI-VM%20Data%20feed/get_api_v1_datafeed
#plot them into map

#load other datasets
#see if we can find in folium an option to switch data on and off (like layers)
#bus api key 	aa059cefd148311cdd0f5c4dd05a48ecd5fc868f
!pip install osmnx
!pip install folium
!pip install requests
api_key = "aa059cefd148311cdd0f5c4dd05a48ecd5fc868f"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 3.6 MB/s eta 0:00:00


In [65]:
# import requests
# import xml.etree.ElementTree as ET

# datafeedID = '8270'
# api_key = "aa059cefd148311cdd0f5c4dd05a48ecd5fc868f"
# base_url = f"https://data.bus-data.dft.gov.uk/api/v1/datafeed/{datafeedID}/?api_key={api_key}"

# try:
#     response = requests.get(base_url)

#     if response.status_code == 200:

#         print(response.content)
#         # Parse the XML response
#         tree = ET.fromstring(response.content)

#         # Now you can navigate through the XML tree to extract the data you need
#         # For example, if you want to get all the 'VehicleMonitoringDelivery' elements:
#         for vm_delivery in tree.findall('.//{http://www.siri.org.uk/siri}VehicleMonitoringDelivery'):
#             # Extract elements from the 'VehicleMonitoringDelivery' here
#             # Example: Get the 'ResponseTimestamp' element
#             response_timestamp = vm_delivery.find('{http://www.siri.org.uk/siri}ResponseTimestamp')
#             if response_timestamp is not None:
#                 print(response_timestamp.text)
#             # Add more parsing as needed based on your requirements

#     else:
#         print(f'Failed to retrieve data: {response.status_code}, Response: {response.text}')

# except requests.exceptions.RequestException as e:
#     print(f'An error occurred: {e}')
import requests
import xml.etree.ElementTree as ET

# Your provided datafeedID and api_key
datafeedID = '8270'
api_key = "aa059cefd148311cdd0f5c4dd05a48ecd5fc868f"

# The base URL for the Bus Location Data API endpoint with the inserted data feed ID and API key
base_url = f"https://data.bus-data.dft.gov.uk/api/v1/datafeed/{datafeedID}/?api_key={api_key}"

# Make a GET request to fetch the data
response = requests.get(base_url)

if response.status_code == 200:

    # Parse the XML response
    tree = ET.fromstring(response.content)

    # Find all elements with the tag 'originRef' within the namespace
    # You will need to use the correct namespace URL if it is not the default one.
    # The namespace is usually defined at the top of the XML document (look for xmlns attribute).
    # This is an example namespace extracted from your previous message. Adjust if necessary.
    namespace = {'ns': 'http://www.siri.org.uk/siri'}

    # Extract all originRef elements
    origin_refs = tree.findall('.//ns:VehicleLocation/ns:originRef', namespaces=namespace)
    print()
    # Print all originRef values
    for origin_ref in origin_refs:
        print(origin_ref)

else:
    print(f'Failed to retrieve data: {response.status_code}, Response: {response.text}')



<Siri xmlns="http://www.siri.org.uk/siri" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.siri.org.uk/siri http://www.siri.org.uk/schema/2.0/xsd/siri.xsd" version="2.0"><ServiceDelivery><ResponseTimestamp>2024-02-10T17:51:55.302820+00:00</ResponseTimestamp><ProducerRef>ItoWorld</ProducerRef><VehicleMonitoringDelivery><ResponseTimestamp>2024-02-10T17:51:55.302820+00:00</ResponseTimestamp><RequestMessageRef>1a798ef2-62eb-4a03-b482-7a5ed5e54a5c</RequestMessageRef><ValidUntil>2024-02-10T17:56:55.302820+00:00</ValidUntil><ShortestPossibleCycle>PT5S</ShortestPossibleCycle></VehicleMonitoringDelivery></ServiceDelivery></Siri>


In [67]:
import requests

# Your provided datafeedID and api_key
datafeedID = '8270'
api_key = "aa059cefd148311cdd0f5c4dd05a48ecd5fc868f"
originRef_value = 'bstmwpg'  # Replace with the actual originRef you're interested in

# The base URL for the Bus Location Data API endpoint with the inserted data feed ID
base_url = f"https://data.bus-data.dft.gov.uk/api/v1/datafeed/{datafeedID}/"

# Set up the parameters including the originRef
params = {
    'api_key': api_key,
    'originRef': originRef_value
}

try:
    # Make a GET request to fetch the data with the parameters
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Assuming the response is in XML format, but you will need to handle the response according to the actual format
        # If the response is in XML, you would parse it using xml.etree.ElementTree as shown in previous examples
        # If the response is JSON, you would use response.json()
        print(response.text)

    else:
        print(f'Failed to retrieve data: {response.status_code}, Response: {response.text}')

except requests.exceptions.RequestException as e:
    print(f'An error occurred: {e}')


<Siri xmlns="http://www.siri.org.uk/siri" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.siri.org.uk/siri http://www.siri.org.uk/schema/2.0/xsd/siri.xsd" version="2.0"><ServiceDelivery><ResponseTimestamp>2024-02-10T17:58:56.865737+00:00</ResponseTimestamp><ProducerRef>ItoWorld</ProducerRef><VehicleMonitoringDelivery><ResponseTimestamp>2024-02-10T17:58:56.865737+00:00</ResponseTimestamp><RequestMessageRef>ed734fba-7820-4145-9c66-dc34e6d404cd</RequestMessageRef><ValidUntil>2024-02-10T18:03:56.865737+00:00</ValidUntil><ShortestPossibleCycle>PT5S</ShortestPossibleCycle></VehicleMonitoringDelivery></ServiceDelivery></Siri>


In [16]:
import osmnx as ox
import folium

# Set the city name or location
place_name = "Milton Keynes, United Kingdom"
map = folium.Map(location=[52.04172, -0.75583], zoom_start=15)
map


In [ ]:
#add data (markers)
folium.Marker([52.039846393280015, -0.7505210635752713], popup="point plotted", icon=folium.Icon(color="greeen", icon="glyphicon glyphicon-plus")).add_to(map)

map

<ipython-input-11-0c6f4ea0e2c4>:2: UserWarning: color argument of Icon should be one of: {'gray', 'darkred', 'lightgreen', 'pink', 'darkgreen', 'lightblue', 'lightgray', 'purple', 'white', 'green', 'red', 'lightred', 'blue', 'cadetblue', 'darkpurple', 'beige', 'orange', 'darkblue', 'black'}.
  folium.Marker([52.039846393280015, -0.7505210635752713], popup="point plotted", icon=folium.Icon(color="greeen", icon="glyphicon glyphicon-plus")).add_to(map)


In [ ]:
folium.Circle(
    radius=1000,
    color="green",
    location=[52.039846393280015, -0.7505210635752713],
    fill=True).add_to(map)

map